<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#label-identity-hairstyle" data-toc-modified-id="label-identity-hairstyle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>label identity hairstyle</a></span></li><li><span><a href="#Prepare-hairstyle-images" data-toc-modified-id="Prepare-hairstyle-images-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare hairstyle images</a></span></li><li><span><a href="#prepare-hairstyle-manifest" data-toc-modified-id="prepare-hairstyle-manifest-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>prepare hairstyle manifest</a></span></li></ul></div>

In [ ]:
from query.models import Video, FaceIdentity, Identity
from esper.widget import *
from esper.prelude import collect, esper_widget
import pickle
import os
import random

get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

# label identity hairstyle

In [ ]:
identity_hair_dict = {}

In [ ]:
identities = Identity.objects.all()
identity_list = [(i.id, i.name) for i in identities]
identity_list.sort()
# 154

In [ ]:
hair_color_3 = {0: 'black', 1: 'white', 2: 'blond'}
hair_color_5 = {0: 'black', 1: 'white', 2: 'blond', 3: 'brown', 4: 'gray'}
hair_length = {0: 'long', 1: 'medium', 2: 'short', 3: 'bald'} 

In [ ]:
identity_label = [id for id in identity_label if id not in identity_hair_dict]

In [ ]:
# idx += 1
# iid = identity_list[idx][0]
# name = identity_list[idx][1]
# iid = identity_label[idx]

# print(name)
print(iid)
result = qs_to_result(
    FaceIdentity.objects \
    .filter(identity__id=1365) \
    .filter(probability__gt=0.8),
    limit=30)
esper_widget(result)

In [ ]:
'''
{'black' : 0, 'white': 1, 'blond' : 2}, # hair_color_3
{'black' : 0, 'white': 1, 'blond' : 2, 'brown' : 3, 'gray' : 4}, # hair_color_5
{'long' : 0, 'medium' : 1, 'short' : 2, 'bald' : 3} # hair_length
'''


label = identity_hair_dict[iid] = (2,2,0)
print(hair_color_3[label[0]], hair_color_5[label[1]], hair_length[label[2]])
pickle.dump(identity_hair_dict, open('/app/data/identity_hair_dict.pkl', 'wb'))

# Prepare hairstyle images

In [ ]:
faceIdentities = FaceIdentity.objects \
    .filter(identity__name='melania trump') \
    .filter(probability__gt=0.9) \
    .select_related('face__frame__video')

In [ ]:
faceIdentities_sampled = random.sample(list(faceIdentities), 1000)
print("Load %d face identities" % len(faceIdentities_sampled))

In [ ]:
identity_grouped = collect(list(faceIdentities_sampled), lambda identity: identity.face.frame.video.id)
print("Group into %d videos" % len(identity_grouped))

In [ ]:
face_dict = {}
for video_id, fis in identity_grouped.items():
    video = Video.objects.filter(id=video_id)[0]
    face_list = []
    for i in fis:
        face_id = i.face.id
        frame_id = i.face.frame.number
        identity_id = i.identity.id
        x1, y1, x2, y2 = i.face.bbox_x1, i.face.bbox_y1, i.face.bbox_x2, i.face.bbox_y2
        bbox = (x1, y1, x2, y2)
        face_list.append((frame_id, face_id, identity_id, bbox))
    face_list.sort()
    face_dict[video.path] = face_list
print("Preload face bbox done")

In [ ]:
if __name__ == "__main__":

    solve_parallel(face_dict, res_dict_path='/app/result/clothing/fina_dict.pkl', workers=10)

# prepare hairstyle manifest

In [ ]:
img_list = os.listdir('/app/result/clothing/images/')
len(img_list)

In [ ]:
group_by_identity = {}
for name in img_list:
    iid = int(name.split('_')[0])
    if iid not in group_by_identity:
        group_by_identity[iid] = []
    else:
        group_by_identity[iid].append(name)
identity_label = [id for id, img_list in group_by_identity.items() if len(img_list) > 10]
identity_label.sort()

In [ ]:
identity_hair_dict = pickle.load(open('/app/data/identity_hair_dict.pkl', 'rb'))
NUM_PER_ID = 1000
hairstyle_manifest = []
for iid, img_list in group_by_identity.items():
    if len(img_list) > 10 and iid in identity_hair_dict:
        if len(img_list) < NUM_PER_ID:
            img_list_sample = img_list
        else:
            img_list_sample = random.sample(img_list, NUM_PER_ID)
        attrib = identity_hair_dict[iid]
        hairstyle_manifest += [(path, attrib) for path in img_list_sample]
random.shuffle(hairstyle_manifest)
len(hairstyle_manifest)

In [ ]:
pickle.dump(hairstyle_manifest, open('/app/result/clothing/hairstyle_manifest.pkl', 'wb'))